In [8]:
import pandas as pd
import numpy as np
import tqdm
import tpot
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import dask_ml.model_selection

In [9]:
from dask.distributed import Client, Variable

In [10]:
df = pd.read_csv('assets/weatherAUS.csv') #membaca data

In [11]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [12]:
df.dtypes #memastikan tipe data sudah tepat

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object

In [13]:
df['Date'] = df.Date.astype('datetime64[ns]') #mengganti tipe data Date

In [14]:
df.shape

(145460, 23)

In [15]:
train,test=train_test_split(df,train_size=0.8,test_size=0.2)

### Data preparation

#### 1.  Missing value Imputation

In [16]:
# df.dropna(inplace=True) 

In [17]:
train.dtypes

Date             datetime64[ns]
Location                 object
MinTemp                 float64
MaxTemp                 float64
Rainfall                float64
Evaporation             float64
Sunshine                float64
WindGustDir              object
WindGustSpeed           float64
WindDir9am               object
WindDir3pm               object
WindSpeed9am            float64
WindSpeed3pm            float64
Humidity9am             float64
Humidity3pm             float64
Pressure9am             float64
Pressure3pm             float64
Cloud9am                float64
Cloud3pm                float64
Temp9am                 float64
Temp3pm                 float64
RainToday                object
RainTomorrow             object
dtype: object

In [18]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

#membagi dataframe menjadi yang numerical dan categorical, untuk missing value imputation
#imputation dengan knn
num_train_df = train.select_dtypes(include=numerics)
cat_train_df = train.select_dtypes(exclude=numerics)

num_test_df = test.select_dtypes(include=numerics)
cat_test_df = test.select_dtypes(exclude=numerics)

In [19]:
num_train_df.head(2)

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
61848,6.7,25.8,0.0,5.8,11.9,39.0,7.0,31.0,87.0,38.0,1015.1,1010.0,3.0,3.0,14.3,25.3
42042,17.8,26.3,0.2,NaN,NaN,31.0,6.0,17.0,80.0,55.0,1017.6,1016.3,8.0,1.0,21.7,25.3


In [20]:
cat_train_df.head(2)

,Date,Location,WindGustDir,WindDir9am,WindDir3pm,RainToday,RainTomorrow
61848,2010-10-29,Sale,E,E,E,No,No
42042,2016-02-07,Williamtown,SE,NW,SSE,No,No


In [21]:
import sys
# from impyute.imputation.cs import mice

In [22]:
#Numerical Data
#missing values imputation with mean median method to Train Data
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer( strategy='mean') #for median imputation replace 'mean' with 'median'
imp_mean.fit(num_train_df)
imputed_num_train_df = imp_mean.transform(num_train_df)

In [23]:
imputed_num_train_df

array([[ 6.7       , 25.8       ,  0.        , ...,  3.        ,
        14.3       , 25.3       ],
       [17.8       , 26.3       ,  0.2       , ...,  1.        ,
        21.7       , 25.3       ],
       [ 7.2       , 17.6       ,  0.        , ...,  4.50321972,
        14.4       , 16.4       ],
       ...,
       [11.3       , 19.9       ,  1.2       , ...,  5.        ,
        16.2       , 19.4       ],
       [ 5.9       , 24.5       ,  0.        , ...,  4.50321972,
        11.5       , 22.2       ],
       [17.2       , 24.6       ,  0.        , ...,  6.        ,
        21.2       , 22.4       ]])

In [24]:
num_train_df=pd.DataFrame(imputed_num_train_df,columns=num_test_df.columns.to_list())

In [25]:
num_train_df.shape

(116368, 16)

In [26]:
#Numerical Data
#missing values imputation with mean median method to Train Data

imp_mean = SimpleImputer( strategy='mean') #for median imputation replace 'mean' with 'median'
imp_mean.fit(num_test_df)
imputed_num_test_df = imp_mean.transform(num_test_df)

In [27]:
imputed_num_test_df

array([[ 9.2       , 25.        ,  0.        , ...,  4.53690532,
        17.6       , 24.7       ],
       [26.6       , 34.5       ,  0.        , ...,  6.        ,
        30.2       , 31.        ],
       [20.9       , 27.        ,  0.        , ...,  4.53690532,
        21.5       , 26.5       ],
       ...,
       [17.6       , 20.7       ,  3.2       , ...,  7.        ,
        18.4       , 19.3       ],
       [ 8.3       , 22.9       ,  0.        , ...,  4.53690532,
        14.2       , 21.9       ],
       [21.9       , 40.9       ,  0.        , ...,  4.        ,
        33.5       , 40.        ]])

In [28]:
num_test_df=pd.DataFrame(imputed_num_test_df,columns=num_test_df.columns.to_list())
num_test_df.shape

(29092, 16)

In [29]:
#categorical imputation using drop na

In [30]:
print(cat_train_df.isnull().sum().sum())
print(cat_test_df.isnull().sum().sum())

25323
6325


In [31]:
imp_most = SimpleImputer( strategy='most_frequent') #for median imputation replace 'mean' with 'median'
imp_most.fit(cat_train_df)
imputed_cat_train_df = imp_most.transform(cat_train_df)

In [32]:
cat_train_df=pd.DataFrame(imputed_cat_train_df,columns=cat_train_df.columns.to_list())

In [33]:
cat_train_df.isnull().sum().sum()

0

In [34]:
imp_most = SimpleImputer( strategy='most_frequent') #for median imputation replace 'mean' with 'median'
imp_most.fit(cat_test_df)
imputed_cat_test_df = imp_most.transform(cat_test_df)

In [35]:
cat_test_df=pd.DataFrame(imputed_cat_test_df,columns=cat_test_df.columns.to_list())

In [36]:
cat_test_df.isnull().sum().sum()

0

In [37]:
train = pd.concat([cat_train_df,num_train_df],axis=1)
train.shape

(116368, 23)

In [38]:
test = pd.concat([cat_test_df,num_test_df],axis=1)
test.shape

(29092, 23)

#### 2. Feature Encoding 

In [39]:
# cat_df = df.select_dtypes(exclude=numerics)

In [40]:
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

In [41]:
le =LabelEncoder()
label_encoded_train_df = train.copy()
for col in label_encoded_train_df.select_dtypes(include='O').columns:
    label_encoded_train_df[col]=le.fit_transform(label_encoded_train_df[col])

In [42]:
label_encoded_train_df=label_encoded_train_df.drop(columns=['Date'])

In [43]:
label_encoded_train_df.reset_index(drop=True)

,Location,WindGustDir,WindDir9am,WindDir3pm,RainToday,RainTomorrow,MinTemp,MaxTemp,Rainfall,Evaporation,...,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
0,35,0,0,0,0,0,6.7,25.8,0.0,5.800000,...,7.0,31.0,87.0,38.0,1015.1,1010.0,3.000000,3.00000,14.3,25.3
1,45,9,7,10,0,0,17.8,26.3,0.2,5.465034,...,6.0,17.0,80.0,55.0,1017.6,1016.3,8.000000,1.00000,21.7,25.3
2,17,7,7,7,0,0,7.2,17.6,0.0,5.465034,...,19.0,26.0,60.0,69.0,1006.7,1005.5,4.446813,4.50322,14.4,16.4
3,38,1,11,1,0,0,16.5,25.3,0.0,7.400000,...,7.0,31.0,68.0,49.0,1025.7,1021.8,4.000000,1.00000,19.8,23.8
4,48,10,10,10,0,0,15.3,30.8,0.0,13.200000,...,24.0,24.0,60.0,21.0,1021.3,1018.4,0.000000,4.50322,18.1,29.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116363,19,13,14,3,0,0,6.3,13.2,1.0,3.000000,...,24.0,22.0,72.0,65.0,1011.1,1009.5,7.000000,7.00000,8.7,12.5
116364,37,13,13,0,1,1,15.2,19.1,4.0,0.400000,...,17.0,22.0,98.0,90.0,1026.4,1024.1,8.000000,8.00000,15.6,16.7
116365,29,3,6,11,1,0,11.3,19.9,1.2,5.465034,...,7.0,2.0,85.0,61.0,1016.5,1014.9,5.000000,5.00000,16.2,19.4
116366,40,3,11,3,0,0,5.9,24.5,0.0,5.465034,...,13.0,19.0,49.0,28.0,1012.7,1009.2,4.446813,4.50322,11.5,22.2


In [44]:
le =LabelEncoder()
label_encoded_test_df = test.copy()
for col in label_encoded_test_df.select_dtypes(include='O').columns:
    label_encoded_test_df[col]=le.fit_transform(label_encoded_test_df[col])

In [45]:
label_encoded_test_df=label_encoded_test_df.drop(columns=['Date'])

In [46]:
label_encoded_test_df.reset_index(drop=True)

,Location,WindGustDir,WindDir9am,WindDir3pm,RainToday,RainTomorrow,MinTemp,MaxTemp,Rainfall,Evaporation,...,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
0,4,3,5,5,0,0,9.2,25.0,0.0,5.481033,...,13.0,9.0,63.0,40.0,1019.0,1012.2,4.450053,4.536905,17.6,24.7
1,13,3,12,6,0,0,26.6,34.5,0.0,5.800000,...,6.0,20.0,83.0,70.0,1011.2,1008.0,6.000000,6.000000,30.2,31.0
2,34,2,10,2,0,0,20.9,27.0,0.0,3.700000,...,15.0,20.0,75.0,53.0,1015.8,1014.8,4.450053,4.536905,21.5,26.5
3,4,11,12,2,0,0,11.6,22.8,0.0,5.481033,...,15.0,4.0,68.0,54.0,1033.2,1030.6,4.450053,4.536905,17.8,20.7
4,46,3,3,3,0,1,14.1,19.3,0.0,5.481033,...,20.0,13.0,70.0,80.0,1018.4,1016.0,4.450053,4.536905,16.0,18.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29087,29,15,14,15,1,0,8.7,18.2,9.8,5.481033,...,4.0,20.0,69.0,44.0,1024.5,1024.3,3.000000,1.000000,14.3,17.2
29088,2,5,2,3,0,0,12.9,29.9,0.0,5.481033,...,9.0,13.0,75.0,41.0,1015.6,1012.0,7.000000,2.000000,18.4,29.1
29089,27,9,9,9,1,0,17.6,20.7,3.2,5.800000,...,20.0,33.0,94.0,78.0,1016.2,1016.1,8.000000,7.000000,18.4,19.3
29090,41,0,0,9,0,0,8.3,22.9,0.0,5.481033,...,19.0,15.0,49.0,25.0,1024.1,1020.4,4.450053,4.536905,14.2,21.9


In [47]:
# column = np.array(['RainToday','RainTomorrow'])
# for col in column:
#     df[col]=df[col].map({'Yes':1,'No':0})

#### 3. Normalization

In [48]:
from sklearn.preprocessing import MinMaxScaler #normalisasi menggunakan minmax scaler

In [49]:
label_encoded_train_df

,Location,WindGustDir,WindDir9am,WindDir3pm,RainToday,RainTomorrow,MinTemp,MaxTemp,Rainfall,Evaporation,...,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
0,35,0,0,0,0,0,6.7,25.8,0.0,5.800000,...,7.0,31.0,87.0,38.0,1015.1,1010.0,3.000000,3.00000,14.3,25.3
1,45,9,7,10,0,0,17.8,26.3,0.2,5.465034,...,6.0,17.0,80.0,55.0,1017.6,1016.3,8.000000,1.00000,21.7,25.3
2,17,7,7,7,0,0,7.2,17.6,0.0,5.465034,...,19.0,26.0,60.0,69.0,1006.7,1005.5,4.446813,4.50322,14.4,16.4
3,38,1,11,1,0,0,16.5,25.3,0.0,7.400000,...,7.0,31.0,68.0,49.0,1025.7,1021.8,4.000000,1.00000,19.8,23.8
4,48,10,10,10,0,0,15.3,30.8,0.0,13.200000,...,24.0,24.0,60.0,21.0,1021.3,1018.4,0.000000,4.50322,18.1,29.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116363,19,13,14,3,0,0,6.3,13.2,1.0,3.000000,...,24.0,22.0,72.0,65.0,1011.1,1009.5,7.000000,7.00000,8.7,12.5
116364,37,13,13,0,1,1,15.2,19.1,4.0,0.400000,...,17.0,22.0,98.0,90.0,1026.4,1024.1,8.000000,8.00000,15.6,16.7
116365,29,3,6,11,1,0,11.3,19.9,1.2,5.465034,...,7.0,2.0,85.0,61.0,1016.5,1014.9,5.000000,5.00000,16.2,19.4
116366,40,3,11,3,0,0,5.9,24.5,0.0,5.465034,...,13.0,19.0,49.0,28.0,1012.7,1009.2,4.446813,4.50322,11.5,22.2


In [50]:
scaler=MinMaxScaler()
scaled=scaler.fit_transform(label_encoded_train_df)

In [51]:
# import seaborn as sns

In [55]:
scaled

array([[0.72916667, 0.        , 0.        , ..., 0.33333333, 0.4535865 ,
        0.58925144],
       [0.9375    , 0.6       , 0.46666667, ..., 0.11111111, 0.60970464,
        0.58925144],
       [0.35416667, 0.46666667, 0.46666667, ..., 0.50035775, 0.4556962 ,
        0.4184261 ],
       ...,
       [0.60416667, 0.2       , 0.4       , ..., 0.55555556, 0.49367089,
        0.47600768],
       [0.83333333, 0.2       , 0.73333333, ..., 0.50035775, 0.39451477,
        0.52975048],
       [0.9375    , 0.6       , 0.6       , ..., 0.66666667, 0.59915612,
        0.53358925]])

In [56]:
label_encoded_train_df=pd.DataFrame(scaled,columns=label_encoded_train_df.columns.to_list())

In [58]:
scaler=MinMaxScaler()
scaled=scaler.fit_transform(label_encoded_test_df)

In [59]:
scaled

array([[0.08333333, 0.2       , 0.33333333, ..., 0.56711316, 0.53483146,
        0.576     ],
       [0.27083333, 0.2       , 0.8       , ..., 0.75      , 0.81797753,
        0.702     ],
       [0.70833333, 0.13333333, 0.66666667, ..., 0.56711316, 0.62247191,
        0.612     ],
       ...,
       [0.5625    , 0.6       , 0.6       , ..., 0.875     , 0.55280899,
        0.468     ],
       [0.85416667, 0.        , 0.        , ..., 0.56711316, 0.45842697,
        0.52      ],
       [0.875     , 0.86666667, 0.2       , ..., 0.5       , 0.89213483,
        0.882     ]])

In [60]:
label_encoded_test_df=pd.DataFrame(scaled,columns=label_encoded_test_df.columns.to_list())

In [61]:
train=label_encoded_train_df.copy()
test=label_encoded_test_df.copy()

In [53]:
# TP = TPOTClassifier(generations=3,population_size=10,cv=5,n_jobs=-1,config_dict=tpot.config.classifier_config_dict_light,use_dask=True)

In [62]:
train.to_csv('train3.csv', index=False)
test.to_csv('test3.csv', index=False)